## Prerequisites
This notebook requires an experimental version of the cmdty-storage package only published on Test PyPI, plus the packages curves for smoothing the forward curve, and ipysheet which provides the input and output grids.

`
pip install -i https://test.pypi.org/simple/ cmdty-storage==0.1.0a17
pip install curves
pip install ipysheet
`

If this notebook has been downloaded standalone, i.e. without downloading the whole storage repo,
it requires the file [storage_gui.py](https://github.com/cmdty/storage/blob/master/samples/python/storage_gui.py) be in the same directory.


In [1]:
%matplotlib inline
import storage_gui as sg
sg.display_gui()

### To Do
* Buttons to clear data:
    * Ratches.
* Import of data clears existing data first:
    * Ratches.
* Optimise performance of ratchets (very slow at moment due to display updating on each cell change).
* Sort out truncation of labels.
* Interest rate curve, not flat rate.
* Ability to select granularity.
* Display simulation results in charts/grids.
    

In [2]:
sg.test_data_btn()

Button(description='Populate Test Data', style=ButtonStyle())

In [3]:
def create_fwd_input(dates, prices):
    dates_cells = sg.ips.Cell(
        value=dates,
        row_start=0,
        row_end=len(dates) - 1,
        column_start=0,
        column_end=0,
        type='date',
        date_format='YYYY-MM-DD',
        squeeze_row=False,
        squeeze_column=True
    )
    prices_cells = sg.ips.Cell(
            value=prices,
            row_start=0,
            row_end=len(prices) - 1,
            column_start=1,
            column_end=1,
            type='numeric',
            numeric_format='0.00',
            squeeze_row=False,
            squeeze_column=True
        )
    cells = [dates_cells, prices_cells]
    return sg.ips.Sheet(rows=len(dates), columns=2, cells=cells, row_headers=False,
                       column_headers=['fwd_start', 'price'])

#dates = ['2021-01-14', '2021-02-01', '2021-03-01', None, None]
#prices = [25.6, 43.56, 38.77, None, None]

today = sg.date.today()
fwd_prices = [58.89, 61.41, 62.58, 58.9, 43.7, 58.65, 61.45, 56.87]
fwd_prices = fwd_prices + [None] * (sg.num_fwd_rows - len(fwd_prices))
fwd_dates = [(today + sg.timedelta(days=do)).strftime('%Y-%m-%d') for do in [0, 30, 60, 90, 150, 250, 350, 400]]
fwd_dates = fwd_dates + [None] * (sg.num_fwd_rows - len(fwd_prices))


#fwd_sheet = sg.create_fwd_input_sheet(fwd_dates, fwd_prices)
#display(fwd_sheet)
#sg.reset_fwd_input_sheet(fwd_sheet)


In [4]:
fwd_dates

['2021-01-17',
 '2021-02-16',
 '2021-03-18',
 '2021-04-17',
 '2021-06-16',
 '2021-09-24',
 '2022-01-02',
 '2022-02-21']

In [5]:
#dir(sg.fwd_input_sheet.cells[0])
sg.fwd_input_sheet.cells[0].value


['',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '']